Name: Arash Asgari

Student Num: 400201037

## HW 8 section 4


In [68]:
import numpy as np
from tqdm.notebook import tqdm, trange

In [69]:
f = open("/content/access.log", 'r')
raw_data = f.readlines()
print(raw_data[0])

130.185.74.243 - - [22/Jan/2019:04:00:36 +0330] "GET /filter/stexists,p8391,b236?page=1 HTTP/1.1" 200 29999 "-" "Mozilla/5.0 (Windows NT 6.1; rv:42.0) Gecko/20100101 Firefox/42.0" "-"



In [74]:
ip_address_list = [x.split()[0] for x in raw_data]
uniques = list(set(ip_address_list))
print("number of actual distinct ip-addresses in the stream is: ", len(uniques))

number of actual distinct ip-addresses in the stream is:  799


In [71]:
class FlajoletMartin():
    def __init__(self, num_of_hashFunc):
        """
        constructor: returns an instance of the class
        :param num_of_hashFunc: number of hash functions you want to apply to be assured of the validness of the output.
        """
        self.num_of_hashFunc = num_of_hashFunc
        self.hash_func_data = np.random.randint(60, size=(4, num_of_hashFunc)) + 30
        self.hash_bias = np.random.randint(1000, size=(1, num_of_hashFunc)) + 1000
        self.modulo = np.random.randint(5000, size=(1, num_of_hashFunc)) + 5000

    def cal_hash(ip, hash_func, bias, modulo):
        """
        example: ip = [1, 2, 3, 4]
                hash function = [20, 10, 15, 42]
                bias = 10
                modulo = 100
                result = 63
        applies a hash function to an IP-address and returns the result
        :param hash_func: the parameters of hash function
        :param bias: an other parameter of hashing which is added to the result of multiplying hash-func by
         the ip-address.
        :param modulo: after multiplying the ip-address array by hash-func and adding the bias we calculate the modulo
        :return: returns the result of applying a hash function to an ip address
        """
        return (np.dot(ip, hash_func) + bias) % modulo

    def num_of_zeros_on_right(self, num: int):
        """
        finds number of zeros on the right side of the binary representation of the number.
        example: num = 10 -> binary representation: (1010) -> result = 1
        :param num: a number that is input.
        :return: the number of zeros on the right side of the binary representation of the number.
        """
        result = 0
        if num == 0:
            return result
        while (num % 2 == 0):
            result = result + 1
            num = num // 2
        return result

    def apply_hashfuncs_to_data(self, ip):
        """
        applies all hash functions to an ip-address and returns the result
        :param ip: ip address as a numpy array of length 4.
        :return: the result of applying hash functions which is a list of length equal to "number of hash functions"
        """
        fl_res = []
        hash_results = (ip.dot(self.hash_func_data) + self.hash_bias).flatten()
        for i in range(self.num_of_hashFunc):
            mod = hash_results[i] % self.modulo.flatten()[i]
            fl_res.append(self.num_of_zeros_on_right(mod))
        return fl_res

    def str_to_ip(self, raw_ip: str):
        """
        converts an ip-address of type string to a numpy array of shape 4.
        :param raw_ip: ip address of type string
            example: "192.168.1.1"
        :return: ip-address converted to array
            example: [192, 168, 1, 1]
        """
        return np.array([int(x) for x in raw_ip.split('.')])

    def run_flajolet(self, list_of_ip):
        """
        applies the hash functions to entire data by calling the "apply_hashfuncs_to_data" function. finally returns
        the number of samples in the stream using flajolet-martin algorithm. It uses multiple hash functions to ensure 
        that the estimations are as close as possible.
        :param list_of_ip: a list of ip-addresses which are of type string each.
        :return: the number of distinct values in the stream.
        """
        hash_zeros = np.zeros((len(list_of_ip), self.num_of_hashFunc))
        for ind in trange(len(list_of_ip)):
            raw_ip = list_of_ip[ind]
            ip = self.str_to_ip(raw_ip)
            hash_res = self.apply_hashfuncs_to_data(ip)
            hash_zeros[ind, :] = np.array(hash_res)
        flajolet_result = np.median(np.max(hash_zeros, 0))
        return int(2 ** flajolet_result)


The algorithm alternatingly returns 512 and 1024 which means that the number of distinct ip-addressses is between these two values.

In [76]:
fl = FlajoletMartin(100)
estimated_num = fl.run_flajolet(ip_address_list)
print("estimated number of distinct ip-addresses: ", estimated_num)

  0%|          | 0/15000 [00:00<?, ?it/s]

estimated number of distinct ip-addresses:  1024
